This version uses Milvus through Docker Compose so you must have Docker installed to run this notebook (Milvus is spun up via `docker compose up -d` as shown in the block below)

In [64]:
# ! pip install pymilvus milvus langchain sentence-transformers tiktoken octoai-sdk
# docker compose up -d

In [65]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint

In [66]:
from dotenv import load_dotenv
import os

load_dotenv()

OCTOAI_API_TOKEN = os.environ.get("OCTOAI_API_TOKEN")

os.environ["OCTOAI_API_TOKEN"] = os.getenv("OCTOAI_API_TOKEN")

In [67]:
template = """Below is an instruction that describes a task. Write a response that appropriately completes the request.\n Instruction:\n{question}\n Response: """
prompt = PromptTemplate.from_template(template)

In [68]:
llm = OctoAIEndpoint(
    endpoint_url="https://text.octoai.run/v1/chat/completions",
    model_kwargs={
        "model": "mixtral-8x7b-instruct-fp16",
        "max_tokens": 128,
        "presence_penalty": 0,
        "temperature": 0.01,
        "top_p": 0.9,
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant. Keep your responses limited to one short paragraph if possible.",
            },
        ],
    },
)

In [69]:
question = "Who was leonardo davinci?"

llm_chain = LLMChain(prompt=prompt, llm=llm)

print(llm_chain.invoke(question)["text"])

 Leonardo da Vinci (1452-1519) was an Italian polymath who is often regarded as one of the greatest painters in history. He is also celebrated for his technological ingenuity, scientific curiosity, and philosophical wisdom. Da Vinci is widely known for his masterpieces such as 'The Last Supper' and 'Mona Lisa.' As an artist, scientist, mathematician, engineer, inventor, anatomist, geologist, cartographer, botanist, musician, and writer, da Vinci embodied the Renaissance ideal. His thirst for


In [70]:
from langchain_community.embeddings import OctoAIEmbeddings
from langchain_community.vectorstores import Milvus

In [71]:
embeddings = OctoAIEmbeddings(endpoint_url="https://text.octoai.run/v1/embeddings")

In [72]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

from langchain.schema import Document
import os

In [73]:
files = os.listdir("./longerData")

In [74]:
file_texts = []

In [75]:
for file in files:
    with open(f"./longerData/{file}") as f:
        if file == ".DS_Store":
            continue
        file_text = f.read()
    #text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    #    chunk_size=1024, chunk_overlap=256, 
    #)

        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=100,
            chunk_overlap=20,
            length_function=len,
            is_separator_regex=False,
        )
    texts = text_splitter.split_text(file_text)
    for i, chunked_text in enumerate(texts):
        file_texts.append(Document(page_content=chunked_text, 
                metadata={"doc_title": file.split(".")[0], "chunk_num": i}))

FileNotFoundError: [Errno 2] No such file or directory: './data/RealDonaldTrump.txt'

In [ ]:
vector_store = Milvus.from_documents(
    file_texts,
    embedding=embeddings,
    connection_args={"host": "localhost", "port": 19530},
    collection_name="cities"
)

In [ ]:
file_texts[0]

Document(page_content="content\nRepresentatives of the people took a vote on the floor of the United States Congress to say loudly and clearly:\n\nLove is love. \nRight is right.\nJustice is justice.\nThanks to the Inflation Reduction Act, out-of-pocket expenses for prescription drug prices will be capped at no more than $2,000 a year for Medicare recipients.\nWhen hospitals and community centers are threatened and intimidated because they support LGBTQI+ children and families, we have to speak out. We must stop hate and violence.\nThe American people won.\nBig Pharma lost.\n\nMedicare will have the power to negotiate for lower prescription drug prices.\nWe are making the largest investment ever in combating the climate crisis through the Inflation Reduction Act.\nWe’re building an economy from the bottom up and the middle out. \n\nAn economy with good jobs and good wages for the long run.\nI had some MAJOR ANNOUNCEMENTS the last couple of weeks, too…\n\n✔️ Inflation’s easing\n✔️ I jus

In [ ]:
retriever = vector_store.as_retriever()

In [ ]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
chain.invoke("What policy do donald trump support?")

' Based on the provided document, Donald Trump supports policies such as "Buy American & Hire American," building a wall along the U.S.-Mexico border, strengthening voting procedures and investigation of voter fraud, and implementing extreme vetting for immigrants. He also emphasizes the importance of strong borders and expresses concern about Christians being executed in the Middle East. However, it\'s important to note that this document is a collection of tweets from Donald Trump during his time in office, and may not represent his complete policy platform.'

In [ ]:
# Let's make this a bit more fun and showcase the multilingual capabilities of Mixtal which really outshine other open source models

# Our Vector DB is populated with entries from english text - even the embedding model we're using here, GTE-Large
# works best on english text. However Mixtral has good mutlilingual capabilities in French, German, Spanish and Italian.
# So what we'll do is ask the assistant to only answer in french in the system and user prompt. RAG here is performed based on 
# english text, but upon producing the user response, the Mixtral LLM will generate tokens in a different language here (french)
french_llm = OctoAIEndpoint(
    endpoint_url="https://text.octoai.run/v1/chat/completions",
    model_kwargs={
        "model": "mixtral-8x7b-instruct-fp16",
        "max_tokens": 128,
        "presence_penalty": 0,
        "temperature": 0.1,
        "top_p": 0.9,
        "messages": [
            {
                "role": "system",
                "content": "You are a helpful assistant who responds in French and not in English.",
            },
        ],
    },
)

french_template = """Answer the question in French based only on the following context:
{context}

Question: {question}
"""
french_prompt = PromptTemplate.from_template(french_template)

In [ ]:
french_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | french_prompt
    | french_llm
    | StrOutputParser()
)

In [ ]:
fr_1 = french_chain.invoke("How big is the city of Seattle?")

In [ ]:
from pprint import pprint
pprint(fr_1)

(' La ville de Seattle est assez grande avec une population de 749 256 '
 "habitants en 2022. C'est la ville la plus peuplée de l'État de Washington et "
 "de la région du Nord-Ouest Pacifique de l'Amérique du Nord. L'aire "
 "métropolitaine de Seattle compte 4,02 millions d'habitants, ce qui en fait "
 'la 15e plus importante aux États-Unis. La croissance de la population de '
 'Seattle a été rapide, avec une augmentation de 21,1%')
